<a href="https://colab.research.google.com/github/zabique/DFL-Colab/blob/master/Copy_of_ENTAR_Pol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><font color="black" size="-5">.</font></b>
![alt text](https://raw.githubusercontent.com/tg-bomze/ENTAR/master/entar.png)

<b><font color="black" size="+3">Zespół instrumentów neuronowych do przywracania animacji</font></b>

*↓ Otwórz mnie ↓*

##**Krótka instrukcja:**

Aby rozpocząć, kliknij ** Otwórz na placu zabaw ** w lewym górnym rogu:

![alt text](https://d.radikal.ru/d39/2001/c1/4192dfd3a667.jpg)

Następnie kliknięcie nawiasów kwadratowych powoduje wykonanie każdego bloku po kolei, czekając na zakończenie poprzedniego:

![alt text](https://d.radikal.ru/d18/2001/1a/b618f3778f86.jpg)

*Blok jest uważany za kompletny w nawiasach kwadratowych* **[_]** *wąż przestanie działać*

##**Złożył to wszystko razem:**
GitHub: [@tg-bomze](https://github.com/tg-bomze)

Telegram: [@bomze](https://t.me/bomze) 

Twitter: [@tg_bomze](https://twitter.com/tg_bomze)

NA POLSKI TŁUMACZYŁ: [@TEN DEEPFAKE ](https://www.youtube.com/user/zabique) i GOOGLE TRANSLATE

*Na wszystkie pytania pisz w telegramach.*

**Dalej są bloki podzielone na sekcje. Wymagana jest sekcja zerowa. Reszta jest opcjonalna.**

# <b><font color="red" size="+3">0. Przygotowanie do przetwarzania</b>
*↓ Kliknij, aby otworzyć sekcję ↓*

In [0]:
#@title ##**Zainstaluj wszystkie niezbędne biblioteki** { display-mode: "form" }
!pip install youtube_dl
from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline

!rm -rf sample_data
clear_output()

In [0]:
#@title ##**Pobierz wideo** { display-mode: "form" }
#@markdown *Wpisz link do filmu poniżej (na przykład YouTube lub Twitter) lub pozostaw pole ** source_url ** puste (w takim przypadku pojawi się monit o pobranie filmu z komputera).*
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  file_name = "downloaded_video." + fn.split(".")[1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    file_name = "downloaded_video." + fn.split(".")[1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
clear_output()
fps_of_video = int(cv2.VideoCapture("downloaded_video.mp4").get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture("downloaded_video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Pobieranie filmów trwających dłużej niż jedną minutę nie jest zalecane.*

#@markdown *Jeśli podczas wykonywania wystąpi błąd, uruchom ponownie ten blok*

**Możesz wykonać następujące sekcje w dowolnej kolejności (najważniejsze jest wykonanie każdego bloku wewnątrz sekcji jeden po drugim).**

---
# <b><font color="red" size="+3">1. Pokoloruj wideo ([DeOldify](https://github.com/jantic/DeOldify))
*↓ Kliknij, aby otworzyć sekcję ↓*


Автор: [Jason Antic](https://twitter.com/citnaj)

In [0]:
#@title ##**Sklonuj repozytorium i zainstaluj niezbędne zależności** { display-mode: "form" }
!git clone https://github.com/jantic/DeOldify.git
%cd DeOldify
!pip install -r colab_requirements.txt
clear_output()

In [0]:
#@title ##**Pobierz wstępnie przeszkolony model** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
try:
  !rm -rf models/ColorizeVideo_gen.pth
  !gdown https://drive.google.com/uc?id=1-3ONnPYcX9fOqnY-pUKYGJOd6XeFon9X -O ./models/ColorizeVideo_gen.pth
except BaseException:
  !rm -rf models/ColorizeVideo_gen.pth
  !wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth
colorizer = get_video_colorizer()
clear_output()

In [0]:
#@title ##**Pokoloruj wideo** { display-mode: "form" }
#@markdown **21, 35** *- оптимальные показатели*
!rm -rf video
!mkdir 'video'
!mkdir 'video/source'
!cp -r ../video.mp4 video/source/video.mp4

render_factor = 21  #@param {type: "slider", min: 5, max: 44}
video_path = colorizer.colorize_from_file_name('video.mp4', render_factor)
clear_output()

Jeśli wystąpi błąd podczas wykonywania bloku z góry, przekonwertuj wideo [ТУТ](https://convert-video-online.com/pl/) i zacznij cały skrypt od nowa.

In [0]:
#@title ##**Otrzymujemy wynik** { display-mode: "form" }
#@markdown **play** *- odtwarzaj wideo w przeglądarce,*

#@markdown **download** *- pobierz na komputer*
!cp -r video/result/video.mp4 ../colorized_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../colorized_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('../colorized_video.mp4')

In [0]:
#@title ###**Opcjonalny blok!** { display-mode: "form" }
#@markdown *Jeśli wydaje ci się, że jakość kolorowania jest niewystarczająca, uruchom ten blok, poczekaj, aż będzie kompletny i sprawdź, jaką wartość * ** render_factor ** * bardziej podoba ci się obraz. Następnie ponownie uruchom poprzedni blok kolorowania, ustawiając żądaną wartość.*
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))

In [0]:
#@title ##**Wyjdź z kolorowania wideo** { display-mode: "form" }
#@markdown *Jeśli zamierzasz nadal wykonywać następujące czynności, to przede wszystkim upewnij się, że ukończyłeś ten blok!*
!rm -rf ../video.mp4
!cp -r video/result/video.mp4 ../video.mp4
!cp -r video/result/video.mp4 ../colorized_video.mp4
%cd ..
clear_output()


---
# <b><font color="red" size="+3">2. Usuwanie drobnych zakłóceń ([DeepRemaster](https://github.com/satoshiiizuka/siggraphasia2019_remastering))
*↓ Kliknij, aby otworzyć sekcję ↓*

Автор: [Satoshi Iizuka](http://iizuka.cs.tsukuba.ac.jp/index_eng.html)

In [0]:
#@title ##**Sklonuj repozytorium** { display-mode: "form" }
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r video.mp4 DeepRemaster/
%cd DeepRemaster
clear_output()

In [0]:
#@title ##**Pobierz wstępnie przeszkolony model** { display-mode: "form" }
try:
  !rm -rf model/remasternet.pth.tar
  !gdown https://drive.google.com/uc?id=1pB8C7c2EDUXNL0Ytk0NZ8q8bAj_nFykO
  !gdown https://drive.google.com/uc?id=1fJJK6i6jVGJmWhoHssuHMZFmfnULxe8S -O model/remasternet.pth.tar
  !bash ./checking_model.sh
except BaseException:
  !rm -rf model/remasternet.pth.tar
  !bash ./download_model.sh
clear_output()

In [0]:
#@title ##**Usuń szum ramy** { display-mode: "form" }
!python remaster.py --input video.mp4 --disable_colorization --gpu
clear_output()

In [0]:
#@title ##**Otrzymujemy wynik** { display-mode: "form" }
#@markdown **play** *- odtwarzaj wideo w przeglądarce,*

#@markdown **download** *- pobierz na komputer*
!rm -rf ../video.mp4
!cp -r video_out.mp4 ../video.mp4
!cp -r video_out.mp4 ../denoise_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../denoise_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('../denoise_video.mp4')

In [0]:
#@title ##**Wyjdź z sekcji Usuń małe zakłócenia** { display-mode: "form" }
#@markdown *Jeśli zamierzasz nadal wykonywać następujące czynności, to przede wszystkim upewnij się, że ukończyłeś ten blok!*
!cp -r video_out.mp4 ../video.mp4
!cp -r video_out.mp4 ../denoise_video.mp4
%cd ..
clear_output()


---
# <b><font color="red" size="+3">3. Interpolacja klatek ([VFIASC](https://github.com/sniklaus/sepconv-slomo))

*(Inną metodę interpolacji można znaleźć w sekcji „Dodatkowe sekcje” poniżej. Jest szybsza, ale mniej wydajna)*

*↓ Kliknij, aby otworzyć sekcję ↓*

Автор: [Simon Niklaus](http://sniklaus.com/about/welcome)

In [0]:
#@title ##**Klonujemy repozytorium i pobieramy niezbędne komponenty** { display-mode: "form" }
!git clone https://github.com/sniklaus/sepconv-slomo.git VFIASC
%cd VFIASC
!rm -rf network-l1.pytorch
!rm -rf network-lf.pytorch
try:
  !gdown https://drive.google.com/uc?id=1v77wNU8sYh0hBmljeGt5LzY9ehQU0elv
  !gdown https://drive.google.com/uc?id=1na11Ey0TB1KEDps4uEwyTBsDcmpo2kr6
except BaseException:
  !bash download.bash
clear_output()

In [0]:
#@title ##**Zainstaluj niezbędne zależności** { display-mode: "form" }
#@markdown **Ten blok jest wymagany. Czas realizacji: ~ 12 minut**
print('Możesz iść na kawę.\n')
!pip install cupy
!pip install moviepy
clear_output()
print('Cóż, to wszystko! Czas zrobić następny blok.')

In [0]:
#@title ##**Interpoluj ramki** { display-mode: "form" }
#@markdown **Ile razy wzrasta FPS:**
fps_boosting = 'x4' #@param ["x4", "x16"]

if fps_boosting == 'x4':
  !python run.py --model lf --video ../video.mp4 --out ./video_x4.mp4
  video_name = "video_x4.mp4"
elif fps_boosting == 'x16':
  !python run.py --model lf --video ../video.mp4 --out ./video_x4.mp4
  !python run.py --model lf --video ./video_x4.mp4 --out ./video_x16.mp4
  video_name = "video_x16.mp4"
clear_output()

mfps_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FPS))
mframes_of_video = int(cv2.VideoCapture(video_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Zaleca się instalację* **x4.** *W przeciwnym razie wykonanie tego bloku może zostać opóźnione lub doprowadzić do błędu. Ponadto efekt SlowMo będzie nienaturalnie silny*

In [0]:
#@title ##**Otrzymujemy wynik** { display-mode: "form" }
#@markdown **play** *- odtwarzaj wideo w przeglądarce,*

#@markdown **download** *- Ściągnij na komputer*
!rm -rf ../video.mp4
!cp -r $video_name ../video.mp4
!cp -r $video_name ../fps_boosted_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../fps_boosted_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("../fps_boosted_video.mp4")

In [0]:
#@title ##**Konwertujemy wideo SlowMo na wysokie częstotliwości i automatycznie sugerujemy pobranie ich na komputer** { display-mode: "form" }
!rm -rf output.mp4
!rm -rf slowed_movie_frames
!rm -rf ../video.mp4
!rm -rf ../fps_boosted_video.mp4

fr = int(fps_of_video*int(fps_boosting.split("x")[1]))
!mkdir 'slowed_movie_frames'
vidcap = cv2.VideoCapture(video_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("slowed_movie_frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

staffs = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  staffs.append("slowed_movie_frames/"+i)

staff = cv2.imread(staffs[0])  # get size from the 1st frame
writer = cv2.VideoWriter(
    'output.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fr,  # fps
    (staff.shape[1], staff.shape[0]),  # width, height
    isColor=len(staff.shape) > 2)
for staff in map(cv2.imread, staffs):
    writer.write(staff)
writer.release()
clear_output()

pfps_of_video = int(cv2.VideoCapture("output.mp4").get(cv2.CAP_PROP_FPS))
pframes_of_video = int(cv2.VideoCapture("output.mp4").get(cv2.CAP_PROP_FRAME_COUNT))

!cp -r output.mp4 ../fps_boosted_video.mp4
!cp -r output.mp4 ../video.mp4
files.download("../fps_boosted_video.mp4")

In [0]:
#@title ##**Wyjdź z interpolacji ramki** { display-mode: "form" }
#@markdown *Jeśli zamierzasz nadal wykonywać następujące czynności, to przede wszystkim upewnij się, że ukończyłeś ten blok!*
%cd ..
clear_output()


---
# <b><font color="red" size="+3">4. Wysoka rozdzielczość ([ESRGAN](https://github.com/xinntao/ESRGAN))

*↓ Kliknij, aby otworzyć sekcję ↓*

Автор: [Xintao Wang](https://xinntao.github.io)

In [0]:
#@title ##**Klonujemy repozytorium i pobieramy niezbędne komponenty** { display-mode: "form" }
!git clone https://github.com/xinntao/ESRGAN.git
!cp -r video.mp4 ESRGAN/
%cd ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames
!rm -rf results/baboon_ESRGAN.png
clear_output()

In [0]:
#@title ##**Podziel wideo na klatki** { display-mode: "form" }
frames_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [0]:
#@title ##**Zwiększ rozdzielczość i popraw jakość klatek** { display-mode: "form" }
#@markdown **Ile razy zwiększyć rozdzielczość:**
upscale = 4 #@param {type: "slider", min: 4, max: 8}

device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    clear_output()
    print("Обработано: {} из {}".format(str(count_frames), str(frames_of_video)))
clear_output()

In [0]:
#@title ##**Zbieranie klatek w filmie** { display-mode: "form" }
frames = []
img = os.listdir("results/")
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscale_video.mp4", frames, fps=fps_of_video)

print('Montaż zakończony')
!cp -r upscale_video.mp4 ../upscale_video.mp4
!cp -r upscale_video.mp4 ../video.mp4
clear_output()

In [0]:
#@title ##**Otrzymujemy wynik** { display-mode: "form" }
#@markdown **play** *- odtwarzać wideo w przeglądarce,*

#@markdown **download** *- Ściągnij na komputer*
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../upscale_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("../upscale_video.mp4")

In [0]:
#@title ##**Wyjście z wysokiej rozdzielczości** { display-mode: "form" }
#@markdown *Jeśli zamierzasz nadal wykonywać następujące punkty, to przede wszystkim upewnij się, że ukończyłeś ten blok!*
%cd ..
clear_output()


# <b><font color="black" size="+3">Dodatkowe sekcje
<font color="grey">Sekcje te zawierają sieci neuronowe, które z różnych powodów nie znajdują się na głównej liście narzędzi do przetwarzania wideo.

*↓ Kliknij, aby otworzyć sekcję ↓*

---
## <b><font color="grey" size="+2">Zwiększona rozdzielczość klatki ([Fast-SRGAN](https://github.com/HasnainRaz/Fast-SRGAN))
*Jeśli zdecydujesz się wykonać ten blok, zaleca się, aby zrobić to ambasadorowi sekcji 2. Usuwanie zakłóceń*

*↓ Kliknij, aby otworzyć sekcję ↓*

Автор: [Hasnain Raza](https://github.com/HasnainRaz)

In [0]:
#@title ###**Клонируем репозиторий и устанавливаем необходимые зависимости** { display-mode: "form" }
!git clone https://github.com/HasnainRaz/Fast-SRGAN.git
!cp -r video.mp4 Fast-SRGAN/
%cd Fast-SRGAN
!pip install -r requirements.txt
clear_output()

In [0]:
#@title ###**Генерируем кадры высокого разрешения** { display-mode: "form" }
#@markdown *Данный блок работает весьма медленно,а также качество итоговых изображений весьма спорное, поэтому я и вынес этот раздел в Дополнительные.*
!rm -rf hr_frames
!rm -rf frames
!mkdir 'hr_frames'
!mkdir 'frames'

vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

!python infer.py --image_dir 'frames/' --output_dir 'hr_frames/'
clear_output()

In [0]:
#@title ###**Собираем кадры в единое видео** { display-mode: "form" }

!rm -rf HR_video.mp4
!rm -rf ../HR_video.mp4
frames = []
img = os.listdir("hr_frames/")
img.sort()
for i in img:
  frames.append(imageio.imread("hr_frames/"+i))
frames = np.array(frames)
imageio.mimsave("HR_video.mp4", frames)

print('Сборка завершена')
!cp -r HR_video.mp4 ../HR_video.mp4
clear_output()

In [0]:
#@title ###**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("HR_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("HR_video.mp4")

In [0]:
#@title ###**Конвертируем СлоуМо видео в высокочастотное и автоматически предлагаем скачать его на компьютер** { display-mode: "form" }
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))

!rm -rf HR_video.mp4
!rm -rf ../HR_video.mp4
frames = []
img = os.listdir("hr_frames/")
img.sort()
clear_output()
for i in img:
  frames.append("hr_frames/"+i)
frames = np.array(frames)

frame = cv2.imread(frames[0])  # get size from the 1st frame
writer = cv2.VideoWriter(
    'HR_video.mp4',
    cv2.VideoWriter_fourcc(*'MP4V'),   # codec (*'DIVX', *'MP4V', *'FMP4', *'MJPG', *'XVID', *'MP4S')
    fps_of_video,  # fps
    (frame.shape[1], frame.shape[0]),  # width, height
    isColor=len(frame.shape) > 2)
for frame in map(cv2.imread, frames):
    writer.write(frame)
writer.release()

clear_output()
print('Конвертирование завершено. Ожидайте начала загрузки (при появлении можно нажать Отмена).')
!cp -r HR_video.mp4 ../HR_video.mp4
files.download("HR_video.mp4")
clear_output()

In [0]:
#@title ###**Выход из раздела Увеличения разрешения** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!rm -rf ../video.mp4
!cp -r HR_video.mp4 ../video.mp4
%cd ..
clear_output()


---
## <b><font color="grey" size="+2">Szybsza, ale niższej jakości interpolacja klatek ([VFI-CFT](https://github.com/MortenHannemose/pytorch-vfi-cft))
*↓ Kliknij, aby otworzyć sekcję ↓*

Автор: [Morten Hannemose](https://scholar.google.ru/citations?user=AH58CjUAAAAJ&hl=ru)

In [0]:
#@title ###**Klonujemy repozytorium i pobieramy wstępnie przeszkolony model** { display-mode: "form" }
!git clone https://github.com/MortenHannemose/pytorch-vfi-cft.git VFI-CFT
video = "video_2x_slow.mp4"
try:
  !rm -rf VFI_CFT_weights.pt.gz
  !rm -rf slow_movie.py
  !gdown https://drive.google.com/uc?id=1-5zS1XOhwVXJhD4JDZWmG6plC_IVVg8b
  !gdown https://drive.google.com/uc?id=1YinJeeUbTbDynD6kw6f6h76CAkRqFdHK
except BaseException:
  !rm -rf VFI_CFT_weights.pt.gz
  !gdown https://drive.google.com/uc?id=1rQAi0UXcaMcEo8_l4oSd8vJUVzp2dIIF
clear_output()

In [0]:
#@title ###**Interpoluj klatki** { display-mode: "form" }
#@markdown **Ile razy wzrasta FPS:**
!rm -rf slowed_movie_frames
!rm -rf $video
!rm -rf video.mp4
!cp -r ../video.mp4 ./
fps_boosting = 2 #@param {type:"slider", min:2, max:8, step:1}

!python slow_movie.py -m video.mp4 -f $fps_boosting

video = "video_"+str(fps_boosting)+"x_slow.mp4"
frames = []
img = os.listdir("slowed_movie_frames/")
img.sort()
clear_output()
for i in img:
  frames.append(imageio.imread("slowed_movie_frames/"+i))
frames = np.array(frames)
imageio.mimsave("fps_boosted_video.mp4", frames)
#@markdown *Więcej niż 4 nie jest zalecane. Efekt SlowMo będzie nienaturalnie silny*
clear_output()

nfps_of_video = int(cv2.VideoCapture("fps_boosted_video.mp4").get(cv2.CAP_PROP_FPS))
nframes_of_video = int(cv2.VideoCapture("fps_boosted_video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
nseconds_of_video = int(nframes_of_video/nfps_of_video)
print("Odebrano film jako ostatni: {} секунд".format(str(nseconds_of_video)))
print("Całkowita liczba ramek: {}".format(str(nframes_of_video)))
print("Liczba klatek na sekundę: {} fps".format(str(nfps_of_video)))

In [0]:
#@title ###**Получаем результат** { display-mode: "form" }
#@markdown **play** *- воспроизвести видео в браузере,*

#@markdown **download** *- загрузить на компьютер*
!rm -rf ../video.mp4
!cp -r fps_boosted_video.mp4 ../video.mp4
!cp -r fps_boosted_video.mp4 ../
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("../fps_boosted_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download("../fps_boosted_video.mp4")

In [0]:
#@title ###**Wyjdź z interpolacji klatki** { display-mode: "form" }
#@markdown *Jeśli zamierzasz nadal wykonywać następujące punkty, to przede wszystkim upewnij się, że ukończyłeś ten blok!*
%cd ..
clear_output()


---
## <b><font color="grey" size="+2">Bonus Kolorowanie zdjęcia i efekt paralaksy  ([DeOldify](https://github.com/jantic/DeOldify) + [3D Ken Burns](https://github.com/sniklaus/3d-ken-burns))</b>
Ta sekcja może być wykonana w pierwszej kolejności. Ostatecznie powstałe wideo można przywrócić w innych sekcjach. Ważne jest, aby przed przejściem przez inne sekcje przejść do ** „0. Przygotowanie do przetwarzania” ** i wykonać blok ** „Zainstaluj wszystkie niezbędne biblioteki” **. Ponadto wszystkie pozostałe sekcje będą działały normalnie.

*↓ **Kliknij, aby otworzyć sekcję** ↓*

In [0]:
#@title ###**Prześlij zdjęcie** { display-mode: "form" }
#@markdown *Wpisz link do zdjęcia poniżej lub pozostaw pole ** source_url ** puste (w takim przypadku pojawi się monit o pobranie filmu z komputera).*
!rm -rf sample_data
source_url = 'http://www.worldfocus.ru/_ph/1/444433832.jpg' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  pic_name = "photo." + fn.split(".")[1]
  !mv -f $fn $pic_name

else:
  try:
    pic_name = "photo." + source_url.split('.')[-1]
    !wget $source_url -O $pic_name
  except BaseException:
    print('Coś poszło nie tak. Spróbuj przesłać zdjęcia z komputera.')

!cp -r photo.jpg downloaded_photo.jpg
clear_output()
#@markdown *Jeśli podczas wykonywania wystąpi błąd, uruchom ponownie ten blok*

### <b><font color="green" size="+1">1) DeOldify - Kolorowanie zdjęć</b>
Autor DeOldify: [Jason Antic](https://twitter.com/citnaj)

*↓ Kliknij, aby otworzyć podsekcję ↓*

In [0]:
#@title ###**Клонируем репозиторий и устанавливаем необходимые зависимости** { display-mode: "form" }
import torch
import fastai
!git clone https://github.com/jantic/DeOldify.git DeOldifyPhoto
%cd DeOldifyPhoto
!pip install -r colab_requirements.txt
clear_output()

In [0]:
#@title ###**Качаем предобученную модель** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
try:
  !rm -rf models/ColorizeVideo_gen.pth
  !gdown https://drive.google.com/uc?id=1hJ_Ut8JQ6mxk0W1ChRmBXcju0oiMo1ZU -O ./models/ColorizeArtistic_gen.pth
except BaseException:
  !rm -rf models/ColorizeVideo_gen.pth
  !wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth
colorizer = get_image_colorizer(artistic=True)
clear_output()

In [0]:
#@title ###**Раскрашиваем фото** { display-mode: "form" }
#@markdown **21** *- оптимальный показатель*
!rm -rf ../colorized_photo.jpg
from google.colab import files
render_factor = 21  #@param {type: "slider", min: 5, max: 44}
image_path = colorizer.plot_transformed_image(path='../photo.jpg', render_factor=render_factor, compare=True)
!cp -r result_images/photo.jpg ../colorized_photo.jpg

#@markdown **Автоматическое скачивание:**
download_image = True #@param {type:"boolean"}
if download_image == True:
  files.download("result_images/photo.jpg")

In [0]:
#@title ###**Выход из раздела DeOldify** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!cp -r result_images/photo.jpg ../photo.jpg
%cd ..
clear_output()


### <b><font color="green" size="+1">2) 3D Ken Burns - Efekt paralaksy</b>

Autor 3D Ken Burns: [Simon Niklaus](http://sniklaus.com/about/welcome)

*↓ Kliknij, aby otworzyć podsekcję ↓*

In [0]:
#@title ###**Клонируем репозиторий и качаем необходимые компоненты** { display-mode: "form" }
import os
import imageio
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
%matplotlib inline
os.environ['CUDA_HOME'] = "/usr/local/cuda"
!pip install moviepy
!git clone https://github.com/sniklaus/3d-ken-burns.git 3D-Ken-Burns
%cd 3D-Ken-Burns
!mkdir 'results'
clear_output()

In [0]:
#@title ###**Качаем предобученную модель** { display-mode: "form" }
!rm -rf models/disparity-estimation.pytorch
!rm -rf models/disparity-refinement.pytorch
!rm -rf models/pointcloud-inpainting.pytorch
try:
  !gdown https://drive.google.com/uc?id=1-0M3GAZkiOexs7gqgWuORzqYAp00DeE- -O ./models/disparity-estimation.pytorch
  !gdown https://drive.google.com/uc?id=1-11KRkOKxqLmKOQk69PDMYbaem-3mH04 -O ./models/disparity-refinement.pytorch
  !gdown https://drive.google.com/uc?id=1-1-VjUQHVDwxtLY9xn65c3LpKCeIW6jH -O ./models/pointcloud-inpainting.pytorch
except BaseException:
  !wget -O ./3d-ken-burns/models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
  !wget -O ./3d-ken-burns/models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
  !wget -O ./3d-ken-burns/models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch
clear_output()

In [0]:
#@title ###**Создаем Параллакс эффект** { display-mode: "form" }
!cp -r  ../photo.jpg ./images/photo.jpg
!rm images/README.md
!rm images/doublestrike.jpg
!for image in ./images/*; do python autozoom.py --in $image --out ./results/$(basename $image | cut -f1 -d '.').mp4; done
clear_output()
#@markdown *Чтобы сохранить видео нажмите по нему правой кнопкой мыши и выберите* **Сохранить**
display(mpy.ipython_display("results/photo.mp4", height=400, autoplay=1, loop=1))

In [0]:
#@title ###**Выход из раздела 3D Ken Burns** { display-mode: "form" }
#@markdown *Если вы собираетесь продолжить выполнять следующие пункты, то прежде всего обязательно выполните этот блок!*
!cp -r results/photo.mp4 ../parallax_video.mp4
!cp -r results/photo.mp4 ../video.mp4
%cd ..
clear_output()